In [1]:
import numpy as np
import pandas as pd
import re
import urllib.request
import requests
from bs4 import BeautifulSoup
import json
import time
from pathlib import Path
import random
import html2text

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as exp_cond
from selenium.common.exceptions import TimeoutException

print('Modules loaded!')

Modules loaded!


In [2]:
def queryString(param):
    return re.sub('[^A-z0-9 ]', '', param).lower().replace(" ", "+").replace("_", "+")

def getCleanData(param):
    result = removeSpecialCharacters(param)
    result = removeStopWords(result)
    return result.lower()

def removeSpecialCharacters(param):
    result = param.replace("-", " ")
    result = re.sub('[^A-z0-9 ]', ' ', result)
    result = ' '.join(result.split())
    return result.lower()

def addSpaceBeforeParentheses(param):
    return re.sub('\(+', ' (', param).replace('  ', ' ')

def getUserAgents():
    l = []
    fp = open('data/user-agents.txt', 'r')

    line  = fp.readline().strip('\n')
    while(line):
        l.append(line)
        line = fp.readline().strip('\n')
    fp.close()
    
    return l

def getHeaders(base_url='https://www.google.co.in/'):
    user_agent = user_agents[random.randint(0, len(user_agents)-1)]         
    headers = {'user-agent': user_agent, 'connection': 'keep-alive', 
               'Accept-Encoding': 'gzip', 'referer': base_url}
    return headers

def randomSleep(minimum, maximum):
    sleeptime =  random.randint(minimum, maximum)
    time.sleep(sleeptime)

def getHTMLData(url, base_url='', number_of_retries=3):
    retries = 1
    response = False
    results = ''
    while retries <= number_of_retries:
        try:
            headers = getHeaders()            
            results = requests.get(url, headers=headers)
            response = True
            break
        except:
            retries += 1
            randomSleep(30, 90)
            
            
    randomSleep(5, 10)
    return response, results

def checkSongArtistExists():
    if songs.shape[0] == 0:
        return 0
    else:
        return songs.loc[((songs['song'] == row['song']) & \
            (songs['artist_name'] == row['artist_name']))].shape[0]
    
def performSearch(search_text):
    driver.find_element_by_id('lst-ib').clear()
    driver.find_element_by_id('lst-ib').send_keys(search_text)
    driver.find_element_by_id('lst-ib').send_keys(Keys.RETURN)
    randomSleep(5, 15)

def getSerachKeyword(song, artist_name):
    position = random.randint(1, 3)
    text = 'lyrics'
    if position == 1:
        keyword = text + ' ' + song + ' ' + artist_name
    if position == 2:
        keyword = song + ' ' + text + ' ' + artist_name
    else:
        keyword = song + ' ' + artist_name + ' ' + text
    return keyword
"""    
def getLyricsLinkFromGoogle():
    name = url = ''

    soup = BeautifulSoup(driver.page_source, "html.parser").find(id = 'search')
    
    try:
        for link in soup.find_all("div", class_="g"):
            if link.div:
                if link.div.div:
                    if link.div.div.h3:
                        if link.div.div.h3.a:
                            url = link.div.div.h3.a.get('href')
                            name = link.div.div.h3.a.get_text()
                            if 'azlyrics.com/lyrics/' in url:
                                return True, url, name.strip()
                            elif 'lyrics.az/' in url:
                                return True, url, name.strip()
                            elif 'genius.com/' in url:
                                return True, url, name.strip()
                            elif 'metrolyrics.com/' in url:
                                return True, url, name.strip()
                            elif 'songtexte.com/songtext/' in url:
                                return True, url, name.strip()
                            elif 'allthelyrics.com/lyrics/' in url:
                                return True, url, name.strip()
                            elif 'azlyricdb.com/lyrics/' in url:
                                url = url + '#.W0UNGtgzZPt'
                                return True, url, name.strip()
    except:
        name = url = ''
    
    return False, url, name
"""
def validateLinks(url):
    response = False
    for index, row in songsites.iterrows():
        text = row['additional_text']
        if row['site'].strip() in url:
            response = True
            if not pd.isna(text):
                url = url.strip() + text.strip()
    return response, url

def getLyricsLinksFromGoogle(keyword):
    performSearch(keyword)
    
    songlinks = pd.DataFrame(columns=['search_title', 'search_url'])
    
    soup = BeautifulSoup(driver.page_source, "html.parser").find(id = 'search')
    
    idx = 0
    for link in soup.find_all("div", class_="g"):
        if link.div:
            if link.div.div:
                if link.div.div.h3:
                    if link.div.div.h3.a:
                        url = link.div.div.h3.a.get('href')
                        name = link.div.div.h3.a.get_text()
                        (response, url) = validateLinks(url)
                        
                        if response:
                            songlinks.loc[idx] = [name, url]
                            idx += 1
    return songlinks

def getLyticsFromAZLyrics(soup):
    try:
        title = soup.find_all('div', class_='col-xs-12 col-lg-8 text-center')[0] \
            .find_all('b')[1].text.strip()
        lyric = soup.find_all('div', class_='col-xs-12 col-lg-8 text-center')[0] \
            .find_all('div')[6].text.strip()
        response = True
    except:
        title = lyrics = ''
        response = False
    return response, title, lyric

def getLyticsFromLyricsAZ(soup):
    try:
        title = soup.find_all('div', class_='right-column main-content lyrics-page')[0]\
            .div.h1.text.strip()
        lyric = soup.find_all('div', class_='right-column main-content lyrics-page')[0]\
            .find_all('p', class_='lyric-text')[0].get_text()
        response = True
    except:
        title = lyric = ''
        response = False
    return response, title, lyric

def getLyticsFromGenius(soup):
    try:
        title = soup.find_all('div', class_='header_with_cover_art-primary_info')[0]\
            .h1.text.strip()
        lyric = soup.find_all('div', class_='song_body-lyrics')[0]\
            .find_all('div', class_='lyrics')[0].get_text()
        response = True
    except:
        title = lyric = ''
        response = False
    return response, title, lyric

def getLyticsFromMetroLyrics(soup):
    try:
        title = soup.find_all('div', class_='banner-heading')[0].h1.text.strip()
        lyric = soup.find_all('div', id='lyrics-body-text')[0].get_text()
        response = True
    except:
        title = lyric = ''
        response = False
    return response, title, lyric

def getLyticsFromSongtexte(soup):
    try:
        title = soup.find_all('div', class_='lyricsContainer')[0].h2.text.strip()
        lyric = soup.find_all('div', id='lyrics')[0].get_text()
        response = True
    except:
        title = lyric = ''
        response = False
    return response, title, lyric

def getLyticsFromAllthelyrics(soup):
    try:
        title = soup.find_all('div', class_='clear-block')[2].h1.text.strip()
        lyric = soup.find_all('div', class_='content-text-inner')[0].get_text().strip()
        response = True
    except:
        title = lyric = ''
        response = False
    return response, title, lyric

def getLyticsFromAZlyricdb(soup):
    try:
        title = soup.find_all('div', id='dl')[0].h1.text.strip()
        lyric = soup.find_all('div', id='lrc')[0].get_text().strip()
        response = True
    except:
        title = lyric = ''
        response = False
    return response, title, lyric

def getLyrics(songlinks):
    title = lyric = url = name = ''
    response = False
        
    for index, row in songlinks.iterrows():
        url = row['search_url']
        name = row['search_title']
        
        (response, results) = getHTMLData(url, number_of_retries=1)
        
        if response:
            soup = BeautifulSoup(results.text, "html.parser")

            if 'azlyrics.com/lyrics/' in url:
                (response, title, lyric) = getLyticsFromAZLyrics(soup)
            elif 'lyrics.az/' in url:
                (response, title, lyric) = getLyticsFromLyricsAZ(soup)
            elif 'genius.com/' in url:
                (response, title, lyric) = getLyticsFromGenius(soup)
            elif 'metrolyrics.com/' in url:
                (response, title, lyric) = getLyticsFromMetroLyrics(soup)
            elif 'songtexte.com/songtext/' in url:
                (response, title, lyric) = getLyticsFromSongtexte(soup)
            elif 'allthelyrics.com/lyrics/' in url:
                (response, title, lyric) = getLyticsFromAllthelyrics(soup)
            elif 'azlyricdb.com/lyrics/' in url:
                (response, title, lyric) = getLyticsFromAZlyricdb(soup)

            if response:
                break

    return response, url, name, title, lyric

print('Functions loaded!')

Functions loaded!


In [3]:
bb_hot100_song = pd.read_csv('data/bb_hot100_song.csv')
bb_hot100_song['processed'] = 0
bb_hot100_song['search_title'] = ''
bb_hot100_song['search_url'] = ''
bb_hot100_song['song_title'] = ''
bb_hot100_song['song_lyric'] = ''

if Path('data/songs.csv').is_file():
    songs = pd.read_csv('data/songs.csv', \
            dtype={'search_title': np.object, \
                   'search_url': np.object, \
                   'song_title': np.object, \
                   'song_lyric': np.object})
else:
    songs = pd.DataFrame(columns=['song', 'artist_name', 'processed', 'search_title', \
                                  'search_url', 'song_title', 'song_info', 'song_lyric'])

songsites = pd.read_csv('data/songsites.csv', \
            dtype={'site': np.object, \
                   'additional_text': np.object})

print('Data loaded!')

Data loaded!


In [ ]:
for index, row in bb_hot100_song.iterrows():
    if checkSongArtistExists() == 0:
        songs = songs.append(row, ignore_index=True)

songs.to_csv('data/songs.csv', encoding='utf-8', index=False)

print('Check and update missing songs')

In [17]:
songs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8357 entries, 0 to 8356
Data columns (total 7 columns):
song            8357 non-null object
artist_name     8357 non-null object
processed       8357 non-null int64
search_title    8316 non-null object
search_url      8316 non-null object
song_title      8316 non-null object
song_lyric      8316 non-null object
dtypes: int64(1), object(6)
memory usage: 457.1+ KB


In [9]:
# Specifying incognito mode as you launch your browser[OPTIONAL]
option = webdriver.ChromeOptions()
option.add_argument("--incognito")

# Create new Instance of Chrome in incognito mode
driver = webdriver.Chrome(executable_path='driver/chromedriver', chrome_options=option)
driver.implicitly_wait(3)
driver.get('http://www.google.com')

print('Initiated web scraping using Selenium!')

Initiated web scraping using Selenium!


In [16]:
index_min, index_max = 7500, 8500

user_agents = getUserAgents()

print('Getting data using Google/Lyrics')
print('----------------')

tot = songs.shape[0]
index_prev = -1

for index, row in songs.iterrows():
    if index >= index_min and index < index_max:
        song = row['song']
        artist_name = row['artist_name']
        processed = row['processed']
        search_title = row['search_title']
        search_url = row['search_url']
        song_title = row['song_title']
        song_lyric = row['song_lyric']

        if processed == 0:
            keyword = getSerachKeyword(song, artist_name)
            songlinks = getLyricsLinksFromGoogle(keyword)
                    
            if songlinks.shape[0] > 0:
                (response, sr_url, sr_title, title, lyric) = getLyrics(songlinks)

                if response:
                    search_title = sr_title
                    search_url = sr_url
                    song_title = title
                    song_lyric = lyric
                    processed = 1

        songs.at[index, 'processed'] = processed
        songs.at[index, 'search_title'] = search_title
        songs.at[index, 'search_url'] = search_url
        songs.at[index, 'song_title'] = song_title
        songs.at[index, 'song_lyric'] = song_lyric
        
        if processed == 0:
            print('Index: ', index, 'Song:', song, ' | Atrist: ', artist_name)
            print('Google: ', search_url)
            print('Title: ', song_title)
            print('-----------------------')
    
songs.to_csv('data/songs.csv', encoding='utf-8', index=False)
print('Processing Done!')

Getting data using Google/Lyrics
----------------
Index:  7984 Song: Corazon  | Atrist:  Maluma X Nego do Borel
Google:  nan
Title:  nan
-----------------------
Index:  8105 Song: Up Down  | Atrist:  Morgan Wallen Featuring Florida Georgia Line
Google:  nan
Title:  nan
-----------------------
Index:  8147 Song: Guatemala  | Atrist:  Swae Lee Featuring Slim Jxmmi
Google:  nan
Title:  nan
-----------------------
Index:  8225 Song: Big Bank  | Atrist:  YG Featuring 2 Chainz, Big Sean & Nicki Minaj
Google:  nan
Title:  nan
-----------------------
Processing Done!


In [ ]:
# songlinks.head()

In [ ]:
# getLyrics(songlinks)

In [15]:
songs.to_csv('data/songs.csv', encoding='utf-8', index=False)

In [ ]:
# getLyrics('http://www.songtexte.com/songtext/marc-nelson/15-minutes-4bdb2726.html')

In [ ]:
# url = 'http://www.songtexte.com/songtext/marc-nelson/15-minutes-4bdb2726.html'
# results = getHTMLData(url)
# soup = BeautifulSoup(results.text, "html.parser")

In [ ]:
# soup\
#                .find_all('div', id='lyrics')[0]\
#                .get_text()

In [ ]:
# soup \
#                .find_all('div', class_='banner-heading')[0] \
#                .h1.text.strip()